## Library


In [53]:
import os
import csv
from typing import List, Dict, Any, Tuple

## Class Process Data


In [54]:
class Transaction:
    """Lớp Transaction chứa thông tin của một giao dịch"""

    def __init__(
        self,
        id: int,
        items: List[int],
        utilities: List[int],
        probabilities: List[float],
    ) -> None:
        """Khởi tạo lớp Transaction

        Parameters
        ----------
        id : int
            ID của giao dịch.
        items : List[int]
            Danh sách các item trong giao dịch.
        utilities : List[int]
            Danh sách các utility của các item trong giao dịch.
        probabilities : List[float]
            Danh sách xác suất của các item trong giao dịch.
        """
        self.id: int = id
        self.items: List[int] = items
        self.utilities: List[int] = utilities
        self.probabilities: List[float] = probabilities

    def __repr__(self) -> Dict[str, Any]:
        """Trả về chuỗi đại diện cho một giao dịch."""
        return str(self.to_dict())

    def __getitem__(self, key: str) -> Any:
        """Trả về giá trị tương ứng với key"""
        return self.to_dict()[key]

    def to_dict(self) -> Dict[str, Any]:
        """Trả về giao dịch dưới dạng dictionary

        Returns
        -------
        Dict[str, Any]
            Dictionary chứa thông tin
        """
        return {
            "id": self.id,
            "items": self.items,
            "utilities": self.utilities,
            "probabilities": self.probabilities,
        }

class UDB:
    """Lớp UDB chứa thông tin của một Uncertainty Database"""

    def __init__(self, file_path: str) -> None:
        """Khởi tạo lớp UDB

        Parameters
        ----------
        file_path : str
            Đường dẫn đến file chứa dữ liệu uncertainty database
        """
        self.file_path: str = file_path
        self.transactions: List[Transaction] = []
        self.read_file()

    def read_file(self) -> None:
        """Đọc tệp dữ liệu uncertainty database (CSV), tạo các đối tượng Transaction, lưu vào danh sách.

        Raises
        ----------
        FileNotFoundError
            Nếu file không tồn tại.
        ValueError
            Nếu có lỗi khi đọc dữ liệu từ file.
        """
        id = 1
        with open(self.file_path, "r") as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                try:
                    # Chuyển đổi các chuỗi thành danh sách
                    items = list(map(int, row["items"].split()))
                    utilities = list(map(int, row["item_utilities"].split()))
                    probabilities = list(map(float, row["item_probabilities"].split()))

                    if len(items) != len(utilities) or len(items) != len(probabilities):
                        raise ValueError(
                            f"Lỗi: Số lượng items, utilities, probabilities không khớp nhau trong dòng: {row}"
                        )

                    transaction = Transaction(
                        id, items, utilities, probabilities
                    )
                    self.transactions.append(transaction)

                    # Tăng ID cho giao dịch tiếp theo
                    id += 1
                except Exception as e:
                    raise ValueError(f"Lỗi xử lý dòng {row}: {e}")

    def get_transactions(self) -> List[Transaction]:
        """Trả về danh sách các giao dịch

        Returns
        ----------
        List[Transaction]
            Danh sách các giao dịch
        """
        return self.transactions


## TKN

In [55]:
class TKNAlgorithm:
    """Thuật toán TKN với các chiến lược cắt tỉa, xác suất và tìm kiếm DFS"""

    def __init__(self, db: UDB, k: int) -> None:
        self.db = db
        self.k = k
        self.min_util = 0  # Ngưỡng Expected Utility tối thiểu
        self.topk_huis = []  # Danh sách Top-k HUIs
        self.transactions = self.db.get_transactions()  # Đọc dữ liệu từ db
        self.twu = {}  # Transaction Weighted Utility (TWU) của các items
        self.existence_map = {}  # Bản đồ tồn tại của các items
        self.pruned_sets = set()  # Lưu các tổ hợp con đã bị loại bỏ

    def calculate_twu_and_existence_map(self) -> None:
        """Tính toán ETWU và bản đồ tồn tại của các items"""
        for transaction in self.transactions:
            for item, utility, probability in zip(transaction['items'], transaction['utilities'], transaction['probabilities']):
                if item not in self.twu:
                    self.twu[item] = 0
                    self.existence_map[item] = set()
                self.twu[item] += utility * probability  # Tích hợp xác suất
                self.existence_map[item].add(transaction['id'])

    def calculate_priu_list(self) -> List[float]:
        """Tính PRIU list (Expected Utility) cho các items"""
        priu_list = []
        for transaction in self.transactions:
            for utility, probability in zip(transaction['utilities'], transaction['probabilities']):
                priu_list.append(utility * probability)  # Lưu Expected Utility
        return sorted(priu_list, reverse=True)

    def apply_priu_strategy(self, priu_list: List[float]) -> None:
        """Áp dụng PRIU strategy để nâng giá trị minUtil"""
        if len(priu_list) >= self.k:
            self.min_util = priu_list[self.k - 1]  # Lấy giá trị lớn thứ k
        else:
            self.min_util = 0  # Nếu PRIU list không đủ k giá trị

    def calculate_expected_utility(self, prefix: List[int], dataset: List[Dict]) -> float:
        """Tính Expected Utility (EU) của một tổ hợp"""
        total_eu = 0.0
        for transaction in dataset:
            for item, utility, probability in zip(transaction['items'], transaction['utilities'], transaction['probabilities']):
                if item in prefix:
                    total_eu += utility * probability  # Tích hợp xác suất vào utility
        return total_eu

    def dfs(self, prefix: List[int], remaining: List[int], existence: set) -> None:
        """Duyệt DFS với tối ưu giao tập tồn tại và cắt tỉa sớm"""
        for i, item in enumerate(remaining):
            new_prefix = prefix + [item]

            # Kiểm tra tổ hợp con đã bị loại trước đó
            if any(set(new_prefix).issuperset(pruned) for pruned in self.pruned_sets):
                continue

            # Giao tập tồn tại
            new_existence = existence & self.existence_map[item]
            if not new_existence:  # Nếu không tồn tại trong giao dịch nào
                continue

            # Tạo tập dữ liệu chiếu
            projected_dataset = self.project_dataset(new_existence)

            # Tính Expected Utility
            eu = self.calculate_expected_utility(new_prefix, projected_dataset)

            # Nếu EU < minUtil, lưu tổ hợp con bị loại
            if eu < self.min_util:
                self.pruned_sets.add(frozenset(new_prefix))
                continue

            # Nếu EU >= minUtil, thêm vào Top-k
            if eu >= self.min_util:
                self.update_top_k(eu, new_prefix)

            # Duyệt tiếp các tổ hợp tiềm năng (sắp xếp động)
            next_items = self.sort_items(remaining[i + 1 :])
            self.dfs(new_prefix, next_items, new_existence)

    def project_dataset(self, transaction_ids: set) -> List[Dict]:
        """Tạo tập dữ liệu chiếu dựa trên transaction IDs"""
        return [
            transaction
            for transaction in self.transactions
            if transaction['id'] in transaction_ids
        ]

    def update_top_k(self, eu: float, itemset: List[int]) -> None:
        """Cập nhật danh sách Top-k HUIs"""
        self.topk_huis.append((eu, itemset))
        self.topk_huis.sort(reverse=True, key=lambda x: x[0])
        if len(self.topk_huis) > self.k:
            self.topk_huis.pop()
            self.min_util = self.topk_huis[-1][0]  # Nâng ngưỡng minUtil lên

    def sort_items(self, remaining_items: List[int]) -> List[int]:
        """Sắp xếp lại các items theo TWU giảm dần"""
        return sorted(remaining_items, key=lambda x: self.twu[x], reverse=True)

    def run(self) -> List[Tuple[float, List[int]]]:
        """Chạy thuật toán với các chiến lược tối ưu"""
        self.calculate_twu_and_existence_map()  # Tính toán ETWU và bản đồ tồn tại
        priu_list = self.calculate_priu_list()  # Tính PRIU list
        self.apply_priu_strategy(priu_list)  # Áp dụng PRIU strategy
        remaining_items = self.sort_items(
            [item for item in self.existence_map.keys() if self.twu[item] >= self.min_util]
        )
        self.dfs([], remaining_items, set(range(1, len(self.transactions) + 1)))
        return self.topk_huis

## ITUFP


In [56]:
# from itertools import combinations

# class ITUFP_Optimized:
#     def __init__(self, udb, k, beam_width=10):
#         """
#         Khởi tạo đối tượng ITUFP tối ưu hóa.
        
#         :param udb: Cơ sở dữ liệu không chắc chắn, là danh sách giao dịch.
#         :param k: Số lượng mẫu cần tìm Top-K.
#         :param beam_width: Độ rộng beam search.
#         """
#         self.udb = udb
#         self.k = k
#         self.beam_width = beam_width
#         self.up_list = {}
#         self.min_sup = 0

#     def expSup(self, itemset):
#         """
#         Tính toán expected support (expSup) cho một itemset.
        
#         :param itemset: Mẫu cần tính toán expSup.
#         :return: Giá trị expSup của itemset.
#         """
#         total = 0
#         for transaction in self.udb:
#             prob = 1
#             for item in itemset:
#                 if item in transaction['items']:
#                     index = transaction['items'].index(item)
#                     prob *= transaction['probabilities'][index]
#                 else:
#                     prob = 0
#                     break
#             total += prob
#         return total

#     def apriori_prune(self, candidates):
#         """
#         Cắt tỉa các tổ hợp không đủ điều kiện dựa trên giá trị min_sup.
        
#         :param candidates: Danh sách các tổ hợp cần kiểm tra.
#         :return: Tập hợp các tổ hợp đủ điều kiện.
#         """
#         pruned_candidates = {}
#         for itemset in candidates:
#             exp_sup = self.expSup(itemset)
#             if exp_sup > self.min_sup:
#                 pruned_candidates[itemset] = exp_sup
#         return pruned_candidates

#     def create_candidates(self, prev_level, size):
#         """
#         Sinh các tổ hợp mới từ các tổ hợp hiện tại.
        
#         :param prev_level: Tập hợp các tổ hợp hiện tại.
#         :param size: Kích thước của tổ hợp mới.
#         :return: Danh sách các tổ hợp mới.
#         """
#         items = set()
#         for itemset in prev_level.keys():
#             items.update(itemset)
#         items = sorted(list(items))
#         return [tuple(sorted(comb)) for comb in combinations(items, size)]

#     def find_top_k(self):
#         """
#         Tìm kiếm Top-K mẫu từ UP-List với kỹ thuật Beam Search.
#         :return: Danh sách các mẫu Top-K.
#         """
#         # Khởi tạo tổ hợp 1-item
#         items = set(item for transaction in self.udb for item in transaction['items'])
#         level = {tuple([item]): self.expSup([item]) for item in items}
#         self.up_list.update(level)

#         size = 1
#         while level:
#             # Chỉ giữ lại Beam Width tổ hợp tốt nhất
#             sorted_level = sorted(level.items(), key=lambda x: x[1], reverse=True)
#             level = dict(sorted_level[:self.beam_width])

#             # Sinh tổ hợp mới từ tổ hợp hiện tại
#             size += 1
#             candidates = self.create_candidates(level, size)

#             # Cắt tỉa các tổ hợp không đủ điều kiện
#             level = self.apriori_prune(candidates)

#             # Thêm các tổ hợp đủ điều kiện vào UP-List
#             self.up_list.update(level)

#             # Nếu đã đủ Top-K, cập nhật min_sup
#             sorted_up_list = sorted(self.up_list.items(), key=lambda x: x[1], reverse=True)
#             if len(sorted_up_list) > self.k:
#                 self.up_list = dict(sorted_up_list[:self.k])
#                 self.min_sup = sorted_up_list[self.k - 1][1]

#     def run(self):
#         """
#         Chạy thuật toán ITUFP tối ưu hóa.
        
#         :return: Top-K mẫu thường xuyên không chắc chắn.
#         """
#         # Tìm Top-K mẫu
#         self.find_top_k()

#         # Sắp xếp kết quả
#         sorted_results = sorted(self.up_list.items(), key=lambda x: x[1], reverse=True)
#         return sorted_results[:self.k]

# if __name__ == "__main__":
#     # Dữ liệu mẫu cho UDB
#     transactions = UDB("./sample_data.csv").get_transactions()

#     # Số mẫu Top-K cần tìm
#     K = 10  # Thay đổi giá trị này theo nhu cầu
    
#     # Khởi tạo đối tượng ITUFP
#     itu_fp = ITUFP_Optimized(transactions, K)
    
#     # Chạy thuật toán ITUFP
#     top_k_result = itu_fp.run()
    
#     # In kết quả Top-K mẫu thường xuyên không chắc chắn
#     print("==" * 20)
#     print("Top-K mẫu thường xuyên không chắc chắn:")
#     for itemset, exp_sup_value in top_k_result:
#         print(f"Itemset: {itemset}, expSup: {exp_sup_value}")

In [57]:
# class ITUFPAlgorithm:
#     def __init__(self, udb: UDB, k: int, beam_width: int = 10) -> None:
#         """ Khởi tạo đối tượng ITUFPAlgorithm tìm kiếm Top-K tập mẫu trong tập dữ liệu không chắc chắn.

#         Parameters
#         ----------
#         udb : UDB
#             Cơ sở dữ liệu không chắc chắn.
#         k : int
#             Số lượng mẫu cần tìm.
#         """
#         self.udb: UDB = udb
#         self.k: int = k
#         self.beam_width: int = beam_width
#         self.up_list: Dict[Tuple[int], float] = {}
#         self.min_sup: float = 0

#     def total_utility(self, itemset: Tuple[int]) -> float:
#         """ Tính tổng utility của một itemset.

#         Parameters
#         ----------
#         itemset : Tuple[int]
#             Mẫu cần tính tổng utility.
#         """
#         total = 0
#         for transaction in self.udb:
#             utility = 0
#             for item in itemset:
#                 if item in transaction['items']:
#                     index = transaction['items'].index(item)
#                     utility += transaction['utilities'][index]

#                 else:
#                     utility = 0
#                     break
#             total += utility
#         return total

#     def apriori_prune(self, candidates: List[Tuple[int]]) -> Dict[Tuple[int], float]:
#         """ Cắt tỉa các tổ hợp không đủ điều kiện dựa trên giá trị min_sup.

#         Parameters
#         ----------
#         candidates : List[Tuple[int]]
#             Danh sách các tổ hợp cần kiểm tra.

#         Returns
#         -------
#         Dict[Tuple[int], float]
#             Tập hợp các tổ hợp đủ điều kiện.
#         """
#         pruned_candidates: Dict[Tuple[int], float] = {}
#         for itemset in candidates:
#             utility = self.total_utility(itemset)
#             if utility > self.min_sup:
#                 pruned_candidates[itemset] = utility
#         return pruned_candidates

#     def generate_combinations(self, items: List[int], size: int, current_comb: List[int] = None, index: int = 0) -> List[Tuple[int]]:
#         """ Sinh tất cả các tổ hợp từ danh sách items với kích thước size.

#         Parameters
#         ----------
#         items : List[int]
#             Danh sách các item để tạo tổ hợp.
#         size : int
#             Kích thước của tổ hợp.
#         current_comb : List[int], optional
#             Tổ hợp hiện tại, mặc định None
#         index : int, optional
#             Vị trí hiện tại, mặc định 0

#         Returns
#         -------
#         List[Tuple[int]]
#             Danh sách các tổ hợp mới.
#         """
#         if current_comb is None:
#             current_comb = []
#         if len(current_comb) == size:
#             return [tuple(current_comb)]
#         if index >= len(items):
#             return []
        
#         # Sinh tổ hợp mới từ tổ hợp hiện tại
#         include_current = self.generate_combinations(items, size, current_comb + [items[index]], index + 1)
        
#         # Bỏ qua item hiện tại
#         exclude_current = self.generate_combinations(items, size, current_comb, index + 1)
        
#         return include_current + exclude_current

#     def create_candidates(self, prev_level: Dict[Tuple[int], float], size: int) -> List[Tuple[int]]:
#         """ Sinh các tổ hợp mới từ các tổ hợp hiện tại.

#         Parameters
#         ----------
#         prev_level : Dict[Tuple[int], float]
#             Tập hợp các tổ hợp hiện tại.
#         size : int
#             Kích thước của tổ hợp mới.

#         Returns
#         -------
#         List[Tuple[int]]
#             Danh sách các tổ hợp mới.
#         """
#         items = set()
#         for itemset in prev_level.keys():
#             items.update(itemset)
#         # Sắp xếp lại các items theo thứ tự tăng dần
#         items = sorted(list(items))
#         return self.generate_combinations(items, size)

#     def find_top_k(self) -> List[Tuple[Tuple[int], float]]:
#         """ Tìm kiếm Top-K mẫu từ UP-List với kỹ thuật Beam Search.

#         Returns
#         -------
#         List[Tuple[Tuple[int], float]]
#             Danh sách các mẫu Top-K.
#         """
#         # Khởi tạo tổ hợp 1-item
#         items = set(item for transaction in self.udb for item in transaction['items'])
#         level = {tuple([item]): self.total_utility([item]) for item in items}
#         self.up_list.update(level)

#         size = 1
#         while level:
#             # Chỉ giữ lại Beam Width tổ hợp tốt nhất
#             sorted_level = sorted(level.items(), key=lambda x: x[1], reverse=True)
#             level = dict(sorted_level[:self.beam_width])

#             # Sinh tổ hợp mới từ tổ hợp hiện tại
#             size += 1
#             candidates = self.create_candidates(level, size)

#             # Cắt tỉa các tổ hợp không đủ điều kiện
#             level = self.apriori_prune(candidates)

#             # Thêm các tổ hợp đủ điều kiện vào UP-List
#             self.up_list.update(level)

#             # Nếu đã đủ Top-K, cập nhật min_sup
#             sorted_up_list = sorted(self.up_list.items(), key=lambda x: x[1], reverse=True)
#             if len(sorted_up_list) > self.k:
#                 self.up_list = dict(sorted_up_list[:self.k])
#                 self.min_sup = sorted_up_list[self.k - 1][1]
    
#     def run(self) -> List[Tuple[Tuple[int], float]]:
#         """ Chạy thuật toán ITUFP để tìm Top-K mẫu thường xuyên không chắc chắn.

#         Returns
#         -------
#         List[Tuple[Tuple[int], float]]
#             Top-K mẫu thường xuyên không chắc chắn.
#         """
#         # Tìm Top-K mẫu
#         self.find_top_k()

#         # Sắp xếp kết quả
#         sorted_results = sorted(self.up_list.items(), key=lambda x: x[1], reverse=True)
#         return sorted_results[:self.k]

# if __name__ == "__main__":
#     transactions = UDB("./test.csv").get_transactions()
#     K = 10
#     itu_fp = ITUFPAlgorithm(transactions, K)
#     top_k_result = itu_fp.run()
#     print("==" * 20)
#     print("Top-K mẫu thường xuyên không chắc chắn:")
#     for itemset, utility in top_k_result:
#         print(f"Itemset: {itemset}, Utility: {utility}")

In [80]:
class UPList:
    """Represents a UP-List structure for storing utilities and probabilities of individual items."""
    def __init__(self, item_name):
        self.item_name = item_name
        self.entries = []  # List of tuples (TID, Probability, Utility)
        self.total_utility = 0.0
        self.exp_support = 0.0

    def add_entry(self, tid, probability, utility):
        """Adds an entry to the UP-List."""
        self.entries.append((tid, probability, utility))
        self.total_utility += utility
        self.exp_support += probability

    def __repr__(self):
        return f"UPList({self.item_name}: {self.entries}, expSup={self.exp_support:.2f}, total_utility={self.total_utility:.2f})"


class IMCUPList:
    """Represents an IMCUP-List for storing combined patterns and their utilities."""
    def __init__(self, pattern_name):
        self.pattern_name = pattern_name
        self.entries = []  # List of tuples (TID, Probability, Utility)
        self.total_utility = 0.0
        self.exp_support = 0.0
        self.index1 = 0
        self.index2 = 0

    def combine(self, up_list1, up_list2):
        """Combines two UP-Lists into this IMCUP-List."""
        idx1, idx2 = 0, 0
        while idx1 < len(up_list1.entries) and idx2 < len(up_list2.entries):
            tid1, prob1, util1 = up_list1.entries[idx1]
            tid2, prob2, util2 = up_list2.entries[idx2]

            if tid1 == tid2:  # Common transaction
                combined_prob = prob1 * prob2
                combined_util = util1 + util2
                self.entries.append((tid1, combined_prob, combined_util))
                self.total_utility += combined_util
                self.exp_support += combined_prob
                idx1 += 1
                idx2 += 1
            elif tid1 < tid2:
                idx1 += 1
            else:
                idx2 += 1

        self.index1 = idx1
        self.index2 = idx2

    def __repr__(self):
        return f"IMCUPList({self.pattern_name}: {self.entries}, expSup={self.exp_support:.2f}, total_utility={self.total_utility:.2f}, Index1={self.index1}, Index2={self.index2})"


class ITUFPVertical:
    """ITUFP algorithm using Vertical Database and UPList structures."""
    def __init__(self, udb: list, k: int):
        self.udb = udb  # List of transactions
        self.k = k  # Number of Top-K patterns
        self.vertical_db = self._build_vertical_db()  # Vertical database using UPLists
        self.top_k = []  # Top-K patterns
        self.min_utility = 0  # Minimum utility threshold

    def _build_vertical_db(self) -> dict:
        """Builds the vertical database using UPLists."""
        vertical_db = {}
        for transaction in self.udb:
            tid = transaction["id"]
            for item, utility, probability in zip(transaction["items"], transaction["utilities"], transaction["probabilities"]):
                if item not in vertical_db:
                    vertical_db[item] = UPList(item)
                vertical_db[item].add_entry(tid, probability, utility)
        return vertical_db

    def _calculate_utility(self, itemset: list) -> int:
        """Calculates the total utility of an itemset using IMCUPList."""
        if len(itemset) == 1:
            return self.vertical_db[itemset[0]].total_utility

        imcup_list = IMCUPList("_".join(map(str, itemset)))
        imcup_list.combine(self.vertical_db[itemset[0]], self.vertical_db[itemset[1]])
        for i in range(2, len(itemset)):
            imcup_list.combine(imcup_list, self.vertical_db[itemset[i]])

        return imcup_list.total_utility

    def _generate_candidates(self, current_level: list) -> list:
        """Generates new candidates from the current level."""
        candidates = []
        for i in range(len(current_level)):
            for j in range(i + 1, len(current_level)):
                if current_level[i][:-1] == current_level[j][:-1]:
                    new_candidate = sorted(set(current_level[i]) | set(current_level[j]))
                    candidates.append(new_candidate)
        return candidates

    def find_top_k(self):
        """Finds the Top-K high utility patterns."""
        current_level = [[item] for item in self.vertical_db.keys()]

        while current_level:
            next_level = []
            for itemset in current_level:
                utility = self._calculate_utility(itemset)
                if utility >= self.min_utility:
                    if len(self.top_k) < self.k or utility > self.min_utility:
                        self.top_k.append((itemset, utility))
                        self.top_k = sorted(self.top_k, key=lambda x: x[1], reverse=True)[:self.k]
                        self.min_utility = self.top_k[-1][1]
                    next_level.append(itemset)

            current_level = self._generate_candidates(next_level)

    def run(self) -> list:
        """Runs the ITUFP algorithm."""
        self.find_top_k()
        return self.top_k


# Example usage
if __name__ == "__main__":
    # Example transactions
    transactions = UDB("./Uncertain_DB/processed_pumsb_utility_spmf.csv").get_transactions()
    # Number of Top-K patterns
    k = 10

    # Run ITUFP
    itu_fp = ITUFPVertical(transactions, k)
    top_k_patterns = itu_fp.run()

    print("Top-K High Utility Patterns:")
    for idx, (itemset, utility) in enumerate(top_k_patterns):
        print(f"{idx + 1}. Itemset: {itemset}, Total Utility: {utility}")


Top-K High Utility Patterns:
1. Itemset: [18, 181, 185, 189, 266, 278, 3405, 4405, 4415, 4429, 4437, 4439, 4941, 7093, 7103], Total Utility: 53631425.0
2. Itemset: [18, 181, 185, 189, 266, 278, 3405, 4405, 4415, 4429, 4439, 4941, 7093, 7103], Total Utility: 49811166.0
3. Itemset: [18, 181, 185, 189, 266, 278, 3405, 4405, 4415, 4429, 4437, 4439, 4941, 7093], Total Utility: 49745598.0
4. Itemset: [18, 181, 185, 189, 266, 278, 3405, 4405, 4415, 4429, 4437, 4941, 7093, 7103], Total Utility: 49634954.0
5. Itemset: [18, 181, 185, 189, 266, 278, 3405, 4405, 4415, 4429, 4437, 4439, 7093, 7103], Total Utility: 49498269.0
6. Itemset: [18, 181, 185, 189, 266, 278, 3405, 4405, 4415, 4429, 4437, 4439, 4941, 7103], Total Utility: 48823198.0
7. Itemset: [18, 181, 185, 189, 266, 278, 3405, 4405, 4415, 4429, 4439, 4941, 7093], Total Utility: 45925339.0
8. Itemset: [18, 181, 185, 189, 266, 278, 3405, 4405, 4415, 4429, 4941, 7093, 7103], Total Utility: 45814695.0
9. Itemset: [18, 181, 185, 189, 266, 278,